# Export Text object to dict

## Structure of the dict representation of the Text object
```python
{
# raw text
'text': ___,
# records of layers in order of dependences
'layers': [
           {
            # simple layer
            'name': ___,
            'attributes': ['attr_1', ...],
            'parent': None,
            'enveloping': None,
            'ambiguous': False,
            'spans': [
                      {'start': ___, 'end': ___, 'attr_1':___, ...},
                      {'start': ___, 'end': ___, 'attr_1':___, ...},
                      ...                     
                     ]
           },
           {
            # parent, not ambiguous
            'name': ___,
            'attributes': ['attr_1', ...],
            'parent': '___',
            'enveloping': None,
            'ambiguous': True,
            'spans': [
                      {'_index_': _, 'attr_1':___, ...},
                      {'_index_': _, 'attr_1':___, ...},
                      ...
                     ]
           },
           {
            # parent, ambiguous
            'name': ___,
            'attributes': ['attr_1', ...],
            'parent': '___',
            'enveloping': None,
            'ambiguous': True,
            'spans': [
                      [{'_index_': _, 'attr_1':___, ...},
                       {'_index_': _, 'attr_1':___, ...},
                       ...
                      ],
                      ...
                     ]
           },
           {
            # enveloping, not ambiguous
            'name': ___,
            'attributes': ['attr_1', ...],
            'parent': None,
            'enveloping': '___',
            'ambiguous': False,
            'spans': [
                      {'_index_': [_, ...], 'attr_1':___, ...},
                      {'_index_': [_, ...], 'attr_1':___, ...},
                      ...
                     ]
           },
           {
            # enveloping, ambiguous
            'name': ___,
            'attributes': ['attr_1', ...],
            'parent': None,
            'enveloping': '___',
            'ambiguous': True,
            'spans': [
                      [
                       {'_index_': [_, ...], 'attr_1':___, ...},
                       {'_index_': [_, ...], 'attr_1':___, ...},
                       ...
                      ],
                      [
                       {'_index_': [_, ...], 'attr_1':___, ...},
                       {'_index_': [_, ...], 'attr_1':___, ...},
                       ...
                      ],
                      ...
                     ]
           }
          ]
}
```

In [1]:
def layer_to_dict(layer, text):
    assert '_index_' not in layer.attributes
    layer_dict = {'name':layer.name,
                  'attributes': layer.attributes,
                  'parent': layer.parent,
                  'enveloping': layer.enveloping,
                  'ambiguous': layer.ambiguous,
                  'spans': []}
    if layer.parent:
        parent_spanlist = text[layer.parent].spans
        records = layer.to_records()
        for span, record in zip(layer, records):
            index = parent_spanlist.index(span)
            if layer.ambiguous:
                for rec in record:
                    rec['_index_'] = index
            else:
                record['_index_'] = index
            layer_dict['spans'].append(record)
    elif layer.enveloping:
        enveloped_spanlist = text[layer.enveloping].spans
        
        records = []
        for spanlist in layer:
            index = [enveloped_spanlist.index(span) for span in spanlist]
            if layer.ambiguous:
                pass
                # TODO:
            else:
                record = {attr:getattr(spanlist, attr) for attr in layer.attributes}
                record['_index_'] = index
                records.append(record)
        layer_dict['spans'] = records
    else:
        layer_dict['spans'] = layer.to_records()
    
    return layer_dict


def export_dict(text):
    text_dict = {'text':text.text, 'layers':[]}
    for layer in text.list_layers():
        text_dict['layers'].append(layer_to_dict(layer, text))
    return text_dict

In [2]:
from estnltk import Text
text = Text('''Mis aias sa-das 2te sorti s-saia? Teine lause.

Teine lõik.''')
text.tag_layer(['morph_analysis', 'paragraphs'])
export_dict(text)

{'layers': [{'ambiguous': False,
   'attributes': (),
   'enveloping': None,
   'name': 'tokens',
   'parent': None,
   'spans': [{'end': 3, 'start': 0},
    {'end': 8, 'start': 4},
    {'end': 11, 'start': 9},
    {'end': 12, 'start': 11},
    {'end': 15, 'start': 12},
    {'end': 19, 'start': 16},
    {'end': 25, 'start': 20},
    {'end': 27, 'start': 26},
    {'end': 28, 'start': 27},
    {'end': 32, 'start': 28},
    {'end': 33, 'start': 32},
    {'end': 39, 'start': 34},
    {'end': 45, 'start': 40},
    {'end': 46, 'start': 45},
    {'end': 53, 'start': 48},
    {'end': 58, 'start': 54},
    {'end': 59, 'start': 58}]},
  {'ambiguous': False,
   'attributes': ['type', 'normalized'],
   'enveloping': 'tokens',
   'name': 'compound_tokens',
   'parent': None,
   'spans': [{'_index_': [2, 3, 4], 'normalized': None, 'type': 'hyphenation'},
    {'_index_': [7, 8, 9], 'normalized': None, 'type': 'hyphenation'}]},
  {'ambiguous': False,
   'attributes': (),
   'enveloping': None,
   'nam